In [1]:
import json, random, os, re
from datetime import datetime

random.seed(42)

SPECIALS = ["<pad>", "<bos>", "<eos>", "<unk>"]

FACTS = [
    ("Türkiye'nin başkenti nedir?", "Ankara"),
    ("İngiltere'nin başkenti nedir?", "Londra"),
    ("Fransa'nın başkenti nedir?", "Paris"),
    ("Almanya'nın başkenti nedir?", "Berlin"),
    ("İtalya'nın başkenti nedir?", "Roma"),
]

SENTIMENT_SENTS = [
    ("bu filmi sevdim harikaydı", "pozitif"),
    ("berbattı hiç beğenmedim", "negatif"),
    ("fena değil idare eder", "nötr"),
    ("müzikler güzel oyunculuk iyi", "pozitif"),
    ("çok sıkıcıydı tavsiye etmem", "negatif"),
]

SMALL_EN_TR = {
    "hello":"merhaba","world":"dünya","cat":"kedi","dog":"köpek","red":"kırmızı",
    "blue":"mavi","green":"yeşil","good":"iyi","bad":"kötü","book":"kitap",
}
SMALL_TR_EN = {v:k for k,v in SMALL_EN_TR.items()}

MONTHS_TR = ["Ocak","Şubat","Mart","Nisan","Mayıs","Haziran","Temmuz","Ağustos","Eylül","Ekim","Kasım","Aralık"]

def norm_date(dmy):
    m = re.match(r"^(\d{1,2})/(\d{1,2})/(\d{4})$", dmy.strip())
    if not m: return "biçim hatası"
    d, mth, y = int(m.group(1)), int(m.group(2)), int(m.group(3))
    if not (1 <= mth <= 12 and 1 <= d <= 31): return "geçersiz tarih"
    return f"{d} {MONTHS_TR[mth-1]} {y}"

def list_task():
    ops = ["sırala", "ters çevir", "benzersiz yap"]
    op = random.choice(ops)
    arr = [random.randint(0,9) for _ in range(random.randint(4,7))]
    if op == "sırala":
        target = " ".join(map(str, sorted(arr)))
    elif op == "ters çevir":
        target = " ".join(map(str, list(reversed(arr))))
    else:
        target = " ".join(map(str, sorted(set(arr))))
    prompt = f"Listeyi {op}: " + " ".join(map(str, arr))
    return prompt, target

def arith_task():
    a = random.randint(0,9)
    b = random.randint(0,9)
    op = random.choice(["+","-","*"])
    if op == "+": ans = a+b
    elif op == "-": ans = a-b
    else: ans = a*b
    return f"Hesapla: {a} {op} {b}", str(ans)

def qa_task():
    return random.choice(FACTS)

def sent_task():
    s, lab = random.choice(SENTIMENT_SENTS)
    return f"Duygu etiketini söyle: {s}", lab

def date_task():
    d = random.randint(1,28)
    m = random.randint(1,12)
    y = random.randint(2015,2024)
    s = f"{d}/{m}/{y}"
    return f"Tarihi normalleştir: {s}", norm_date(s)

def mt_task():
    # mini çeviri, iki yönde
    if random.random() < 0.5:
        en = random.choice(list(SMALL_EN_TR.keys()))
        return f"İngilizceyi Türkçeye çevir: {en}", SMALL_EN_TR[en]
    else:
        tr = random.choice(list(SMALL_TR_EN.keys()))
        return f"Türkçeyi İngilizceye çevir: {tr}", SMALL_TR_EN[tr]

GENS = [qa_task, sent_task, list_task, date_task, mt_task, arith_task]

def make_examples(n):
    data = []
    for _ in range(n):
        p, t = random.choice(GENS)()
        data.append({"input": p, "target": t})
    return data

def build_vocab(examples):
    # whitespace tokenization → kelime listesi
    toks = set(SPECIALS)
    for ex in examples:
        toks.update(ex["input"].strip().split())
        toks.update(ex["target"].strip().split())
    # indexle
    stoi = {tok:i for i,tok in enumerate(sorted(toks))}
    itos = [None]*len(stoi)
    for k,v in stoi.items(): itos[v]=k
    return {"stoi": stoi, "itos": itos}

def save_jsonl(path, rows):
    with open(path, "w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

def main(out_dir=".", n_train=2000, n_dev=200):
    os.makedirs(out_dir, exist_ok=True)
    train = make_examples(n_train)
    dev = make_examples(n_dev)
    vocab = build_vocab(train + dev)

    save_jsonl(os.path.join(out_dir, "train.jsonl"), train)
    save_jsonl(os.path.join(out_dir, "dev.jsonl"), dev)
    with open(os.path.join(out_dir, "vocab.json"), "w", encoding="utf-8") as f:
        json.dump(vocab, f, ensure_ascii=False)

    print(f"✅ train.jsonl: {len(train)}")
    print(f"✅ dev.jsonl  : {len(dev)}")
    print(f"✅ vocab.json : {len(vocab['itos'])} token")

if __name__ == "__main__":
    main()


✅ train.jsonl: 2000
✅ dev.jsonl  : 200
✅ vocab.json : 471 token
